In [1]:
 %run "../Libraries/ADP_Farmatic_Sellout_Process"

In [2]:
process = "FMT_SL"
logger_name = "FMT_Data_TobeProcessed_Sellout"
level_action = "DEFAULT"
log_level = "DEFAULT"

In [3]:
###############################################################################
""" Main Process for Farmatic Canonize Files

"""
 #Who                 When           What
 #Victor Salesa       05/11/2018     Create Process Stub
 #Victor Salesa       10/01/2019     Substituted cache with persist and placed count to force cache generation
 #Victor Salesa       18/01/2019     Added Error control code to stop process if Anything fails
 #Victor Salesa       26/02/2019     Added dbutils.notebook.exit 
 #Ana Perez           06/03/2019     Unpersist Dataframes
 #Ana Perez           28/03/2019     Included log managment and exception managment
 #Victor Salesa       04/04/2019     Included Recovery Mode Parameter and Flow Control to Activate Recovery
 #Victor Salesa       04/04/2019     Added job_id to GenerateCanonicalResult
 #Victor Salesa       21/04/2019     Added StartADPProcess and EndADPProcess with row count Pending to correct tables to write to the right ones.
################################################################################

# Process Landing Files
try:
  
  #Create parameter to launch task from notebook launcher
  try:
    level_action = dbutils.widgets.get("level_action") 
    log_level = dbutils.widgets.get("log_level") 
    recovery_mode = dbutils.widgets.get("recovery_mode")
    ADP_log_info(process, logger_name, level_action, log_level, "BEGIN", sys._getframe().f_code.co_name)   
  except Exception as err:
    ADP_log_info(process, logger_name, "DEFAULT", "DEFAULT", "Internal Call parameter was not Created Please Re-run", sys._getframe().f_code.co_name)   

    dbutils.widgets.text("level_action", "DEFAULT","Level Action")
    dbutils.widgets.text("log_level","DEFAULT","Log Level")
    dbutils.widgets.text("recovery_mode","OFF","Recovery Mode")

    level_action = dbutils.widgets.get("level_action") 
    log_level = dbutils.widgets.get("log_level")
    recovery_mode = dbutils.widgets.get("recovery_mode")

  #end exception 
  
  #Generate Job Id and log result to JOB ID TABLE (PENDING CHANGE TABLE FROM CTL.VSS_JOB TO CTL.JOB)
  job_id = str(StartADPProcess(process))
   
  if recovery_mode != 'ON':
    
    print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+ '---------------------------Begin Sell-Out Process-----------------------------------------------')

    #Read Source files for Sellout and transform to a Normalized Big Distributed File with 
    #all the information in the files in Tabular format 
    NormalizedDF = NormalizeFarmaticSellout(__PHARMATIC_TOBEPROCESSED_BASE_PATH__+"*GSL*.TXT",debug=False,sample=False,sample_quantity=1)

    if NormalizedDF==None:
      raise Exception('Normalize Process Error')

    #Cache Data of Normalized Files 
    NormalizedDF.persist(StorageLevel.MEMORY_AND_DISK)
    NormalizedDF.count()

    print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+ '---------------------------Normalize Data End--------------------------------------------------')

    #Add structure validation fields to the output of the previous transformation
    FieldFormatValidationsDF               = FieldFormatValidationsFarmaticSellout(NormalizedDF,debug=False)

    if NormalizedDF==None:
      raise Exception('FieldFormatValidations Process Error')

    #Cache Data of Validated Files
    FieldFormatValidationsDF.persist(StorageLevel.MEMORY_AND_DISK)
    FieldFormatValidationsDF.count()

    print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+ '---------------------------Field Format Validations End----------------------------------------')

    #Add enrichment fields to the output of the previous transformation
    EnrichmentDF                           = EnrichmentFarmaticSellout(FieldFormatValidationsDF,debug=False)

    if EnrichmentDF==None:
      raise Exception('Enrichment Process Error')

    #Cache Enriched Files
    EnrichmentDF.persist(StorageLevel.MEMORY_AND_DISK)
    EnrichmentDF.count()

    print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+ '---------------------------Enrichment End-----------------------------------------------------')

    #Add enrichment validation fields to the output of the previous transformation
    EnrichmentValidationsDF                = EnrichmentValidationsFarmaticSellout(EnrichmentDF,debug=False)

    #Cache Enriched and Validated Files
    EnrichmentValidationsDF.persist(StorageLevel.MEMORY_AND_DISK)
    EnrichmentValidationsDF.count()

    if EnrichmentValidationsDF==None:
      raise Exception('EnrichmentValidations Process Error')


    print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+ '---------------------------Enrichment Validations End-----------------------------------------')

    TemporaryFarmaticSelloutDF                = GenerateTemporaryFarmaticSellout(EnrichmentValidationsDF,debug=False)
    
    if TemporaryFarmaticSelloutDF==None:
      raise Exception('TemporaryFarmaticSelloutDF Process Error')
    
    #Cache Enriched and Validated Files
    TemporaryFarmaticSelloutDF.persist(StorageLevel.MEMORY_AND_DISK)
    TemporaryFarmaticSelloutDF.count()

    print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+ '---------------------------Temporary file End--------') 
  else:    
    TemporaryFarmaticSelloutDF = RecoverTemporaryFarmaticSellout(debug=False)
    
    if TemporaryFarmaticSelloutDF==None:
      raise Exception('TemporaryFarmaticSelloutDF Process Error')
    
    #Cache Enriched and Validated Files
    TemporaryFarmaticSelloutDF.persist(StorageLevel.MEMORY_AND_DISK)
    TemporaryFarmaticSelloutDF.count()
    
  #end if recovery_mode != 'ON':
  
  #   Generate QA data structure and store in DDBB and file 
  print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+ '---------------------------GenerateQAResult Start-----------------------------------------')

  #Get row count pending change JOB_ID field name from JOB_ID to something like ROW_COUNT
  row_count_after = TemporaryFarmaticSelloutDF.select("NUM_ROWS_TOTAL").distinct().first().NUM_ROWS_TOTAL
  
  GenerateQAResult                       = GenerateQAFarmaticSellout(TemporaryFarmaticSelloutDF,debug=False,job_id=job_id)
  
  if GenerateQAResult==False:
    raise Exception('GenerateQAResult Process Error')
  
  print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+ '---------------------------GenerateQAResult End-----------------------------------------')
   
  #Generate Canonical data structure and store in DDBB and file
  GenerateCanonicalResult                = GenerateCanonicalFarmaticSellout(TemporaryFarmaticSelloutDF,debug=False,job_id=job_id)
  
  if GenerateCanonicalResult==False:
    raise Exception('GenerateCanonicalResult Process Error') 
  
  print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+ '---------------------------GenerateCanonicalResult End-----------------------------------------')
  
  if recovery_mode == 'OFF':
    EnrichmentValidationsDF.unpersist()
    EnrichmentDF.unpersist()
    FieldFormatValidationsDF.unpersist()
    NormalizedDF.unpersist()
  #end if recovery_mode == 'OFF':
  TemporaryFarmaticSelloutDF.unpersist()
  
  EndADPProcess(row_count_after,job_id=int(job_id))
  
  ADP_log_info(process, logger_name, level_action, log_level, "END", sys._getframe().f_code.co_name) 
  
except Exception as err:
  message = ADP_log_exception(process, logger_name, level_action, log_level,  "", sys._getframe().f_code.co_name,  sys.exc_info())
  message = str(message).replace("'","").replace('"','').replace('\\n','\n')
  EndADPProcess(0,job_id=int(job_id),log_error_code=50003,log_error_message=message)
  raise Exception(err)